In [21]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import time
import re
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from kanjize import kanji2number

# --- ⚙️ 設定（ここを自分の環境に合わせて書き換える） ---
AGE_START = 0                  # 検索開始年齢
AGE_END = 100                 # 検索終了年齢（テスト用: 10歳まで）
MAX_RECORDS = 100             # 取得したい件数（テストなら100、本番なら1000〜）
JSON_FILE = "service_account.json"  # ⚠️ GCPからDLしたJSONファイル名
SPREADSHEET_NAME = "bookdata"   # ⚠️ 作成したスプシの名前



# --- 🛠️ 関数定義 ---

def extract_age(title):
    """タイトルから年齢を抽出する関数（漢数字対応版）"""
    try:
        # パターン1: 半角/全角数字 + 歳/才 (例: 13歳, １３才)
        # unicodedataで正規化してもいいが、kanji2intが優秀なので任せる手もある
        # ここではシンプルに数字抽出
        match = re.search(r'(\d+)(?:歳|才)', title)
        if match:
            return int(match.group(1))
        
        # パターン2: 漢数字 + 歳/才 (例: 十三歳)
        match_kanji = re.search(r'([一二三四五六七八九十百]+)(?:歳|才)', title)
        if match_kanji:
            return kanji2number(match_kanji.group(1))
            
    except Exception:
        pass
    return None

def is_valid_age_title(title):
    """タイトルに「〇〇歳からの」というパターンが含まれているかチェック"""
    if not title:
        return False
    
    # パターン1: 数字 + 歳からの (例: 3歳からの、13歳からの)
    if re.search(r'\d+歳からの', title):
        return True
    
    # パターン2: 漢数字 + 歳からの (例: 三歳からの、十三歳からの)
    if re.search(r'[一二三四五六七八九十百]+歳からの', title):
        return True
    
    return False

def classify_genre(title):
    """タイトルからジャンルを推定する関数（キーワードベース）"""
    if not title:
        return "その他"
    
    title_str = str(title).lower()
    
    # ジャンル分類用キーワード辞書（優先度順）
    genre_keywords = {
        "自己啓発・心理学": [
            "自信", "自己", "自分", "心", "思考", "感情", "メンタル", "ストレス", 
            "モチベーション", "やる気", "習慣", "成功", "目標", "夢", "変わる", "変えたい",
            "成長", "生き方", "人生", "幸福", "幸せ", "ポジティブ", "ネガティブ"
        ],
        "教育・学習": [
            "勉強", "学習", "学び", "学習法", "記憶", "暗記", "受験", "テスト", 
            "試験", "学校", "授業", "教育", "教養", "知識"
        ],
        "仕事・ビジネス": [
            "仕事", "ビジネス", "働き方", "キャリア", "就職", "転職", "起業", 
            "会社", "職場", "マネジメント", "リーダー", "チーム"
        ],
        "お金・投資": [
            "お金", "貯金", "節約", "投資", "資産", "株式", "金融", "経済", 
            "ファイナンス", "お金の", "稼ぐ"
        ],
        "健康・スポーツ": [
            "健康", "体", "身体", "運動", "スポーツ", "ダイエット", "食事", "栄養",
            "トレーニング", "筋力", "フィットネス", "ヨガ", "ランニング"
        ],
        "料理・食事": [
            "料理", "レシピ", "クッキング", "パン", "お菓子", "スイーツ", "グルメ", 
            "食", "味", "食材"
        ],
        "アート・デザイン": [
            "アート", "絵", "イラスト", "デザイン", "美術", "アーティスト", 
            "クリエイティブ", "創作"
        ],
        "音楽": [
            "音楽", "ピアノ", "ギター", "楽器", "歌", "ミュージック", "音", "リズム"
        ],
        "科学・数学": [
            "科学", "数学", "物理", "化学", "生物", "実験", "研究", "発見", 
            "宇宙", "自然", "環境"
        ],
        "歴史・社会": [
            "歴史", "戦争", "政治", "社会", "文化", "伝統", "民族", "世界", "日本"
        ],
        "文学・小説": [
            "小説", "物語", "ストーリー", "文学", "詩", "エッセイ", "随筆"
        ],
        "コミック・マンガ": [
            "マンガ", "コミック", "アニメ", "キャラクター"
        ],
        "語学": [
            "英語", "英会話", "言語", "語学", "フランス語", "中国語", "韓国語", 
            "会話", "文法", "単語"
        ],
        "趣味・娯楽": [
            "趣味", "ゲーム", "旅行", "写真", "コレクション", "手芸", "工作"
        ],
        "恋愛・人間関係": [
            "恋愛", "恋", "結婚", "友達", "人間関係", "コミュニケーション", 
            "対人", "友情", "愛情"
        ],
    }
    
    # 各ジャンルについてキーワードマッチング
    for genre, keywords in genre_keywords.items():
        for keyword in keywords:
            if keyword in title_str:
                return genre
    
    return "その他"

def fetch_and_save_books():
    """NDLから取得 → 年齢抽出 → スプシ保存 を一気に行う"""
    
    # 1. API設定
    url = "https://ndlsearch.ndl.go.jp/api/opensearch"
    step = 500 # NDLの1回の上限
    processed_data = []
    seen_titles = set()  # 重複チェック用
    
    print(f"📡 国立国会図書館APIから「〇〇歳の」または「〇〇歳からの」を含む本を収集中...")
    print(f"   検索年齢範囲: {AGE_START}歳〜{AGE_END}歳")
    
    # --- 各年齢について検索 ---
    total_ages = AGE_END - AGE_START + 1
    for age_idx, age in enumerate(range(AGE_START, AGE_END + 1), 1):
        print(f"\n📊 進捗: {age_idx}/{total_ages}歳目 ({age}歳) - 現在の取得数: {len(processed_data)}件")
            
        # 検索キーワードパターン: 「〇〇歳の」と「〇〇歳からの」
        keywords = [f"{age}歳の", f"{age}歳からの"]
        
        for keyword in keywords:
            current_idx = 1
            print(f"  🔍 「{keyword}」で検索中...")
            
            # --- データ取得ループ ---
            while True:
                params = {
                    "title": keyword,
                    "cnt": step,
                    "idx": current_idx
                }
                
                try:
                    res = requests.get(url, params=params, timeout=30)
                    res.raise_for_status()
                    
                    # XMLパース（名前空間を登録）
                    root = ET.fromstring(res.content)
                    # 名前空間の登録
                    namespaces = {
                        'dc': 'http://purl.org/dc/elements/1.1/',
                        'dcterms': 'http://purl.org/dc/terms/',
                        'atom': 'http://www.w3.org/2005/Atom'
                    }
                    items = root.findall(".//item")
                    
                    if not items:
                        break
                    
                    print(f"    -> {current_idx}件目〜 を処理中... (ヒット数: {len(items)})")
                    
                    for item in items:
                        # 指定されたフィールドのみを取得
                        item_data = {}
                        
                        # titleの取得
                        title = ""
                        # パターン1: dc:title
                        dc_title = item.find(".//dc:title", namespaces)
                        if dc_title is not None and dc_title.text:
                            title = dc_title.text.strip()
                        # パターン2: title要素
                        elif item.find("title") is not None and item.find("title").text:
                            title = item.find("title").text.strip()
                        
                        # dateの取得
                        date = ""
                        # パターン1: dc:date
                        dc_date = item.find(".//dc:date", namespaces)
                        if dc_date is not None and dc_date.text:
                            date = dc_date.text.strip()
                        # パターン2: dcterms:issued
                        elif item.find(".//dcterms:issued", namespaces) is not None:
                            dcterms_issued = item.find(".//dcterms:issued", namespaces)
                            if dcterms_issued.text:
                                date = dcterms_issued.text.strip()
                        # パターン3: pubdate
                        elif item.find("pubdate") is not None and item.find("pubdate").text:
                            date = item.find("pubdate").text.strip()
                        
                        # creatorの取得
                        creator = ""
                        # パターン1: dc:creator
                        dc_creators = item.findall(".//dc:creator", namespaces)
                        if dc_creators:
                            creators = [c.text.strip() for c in dc_creators if c.text]
                            creator = " / ".join(creators) if creators else ""
                        # パターン2: author要素
                        elif item.find("author") is not None and item.find("author").text:
                            creator = item.find("author").text.strip()
                        
                        # subjectの取得
                        subject = ""
                        # dc:subject
                        dc_subjects = item.findall(".//dc:subject", namespaces)
                        if dc_subjects:
                            subjects = [s.text.strip() for s in dc_subjects if s.text]
                            subject = " / ".join(subjects) if subjects else ""
                        # フォールバック: subject要素
                        elif item.find("subject") is not None and item.find("subject").text:
                            subject = item.find("subject").text.strip()
                        
                        # タイトルで重複チェック & 「〇〇歳の」または「〇〇歳からの」パターンのチェック
                        if title and title not in seen_titles and is_valid_age_title(title):
                            seen_titles.add(title)
                            # ジャンル分類
                            genre = classify_genre(title)
                            item_data = {
                                "title": title,
                                "date": date,
                                "creator": creator,
                                "subject": subject,
                                "genre": genre
                            }
                            processed_data.append(item_data)
                    
                    current_idx += step
                    
                except Exception as e:
                    print(f"    ⚠️ エラー発生: {e}")
                    break
    
    print(f"✨ 抽出完了: {len(processed_data)} 件の有効データが見つかりました。")

    # --- スプレッドシート保存 ---
    if processed_data:
        print("\n🚀 Googleスプレッドシートに書き込み中...")
        try:
            # 認証
            scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
            creds = ServiceAccountCredentials.from_json_keyfile_name(JSON_FILE, scope)
            client = gspread.authorize(creds)
            
            # シートを開く
            sheet = client.open(SPREADSHEET_NAME).sheet1
            
            # ヘッダー行を作成（日本語の列名）
            headers = ["タイトル", "発行日", "作成者", "主題", "ジャンル"]
            
            # 既存のデータを読み込む（重複チェック用）
            existing_titles = set()
            existing_data = []
            try:
                existing_data = sheet.get_all_values()
                if existing_data:
                    # ヘッダー行を確認
                    if existing_data[0] == headers:
                        # 既存のタイトルを収集（1行目がヘッダーなので2行目から）
                        for row in existing_data[1:]:
                            if row and row[0]:  # タイトル列が存在する場合
                                existing_titles.add(row[0])
                        print(f"   既存データ: {len(existing_titles)}件のタイトルを確認しました。")
                    else:
                        print(f"   既存のヘッダーと異なるため、既存データは無視します。")
            except Exception as e:
                print(f"   既存データの読み込み中にエラー: {e}（新規作成として処理します）")
            
            # 新しいデータのみをフィルタリング
            new_rows = []
            for item in processed_data:
                title = item.get("title", "")
                # 既存データにない場合のみ追加
                if title and title not in existing_titles:
                    row = [
                        item.get("title", ""),
                        item.get("date", ""),
                        item.get("creator", ""),
                        item.get("subject", ""),
                        item.get("genre", "")
                    ]
                    new_rows.append(row)
                    existing_titles.add(title)  # 今回追加するデータも重複チェックに追加
            
            if new_rows:
                # ヘッダーが存在しない場合は追加
                if not existing_data or existing_data[0] != headers:
                    sheet.append_row(headers)
                
                # 新しいデータのみを追加
                sheet.append_rows(new_rows)
                print(f"✅ 書き込み成功！{len(new_rows)}件の新しいデータを追加しました。")
                print(f"   合計: {len(existing_titles)}件のデータがスプレッドシートに保存されています。")
            else:
                print(f"ℹ️  新しいデータはありませんでした（全て既存データと重複）。")
            
            print("   ブラウザでスプレッドシートを確認してください。")
            
        except Exception as e:
            print(f"❌ スプレッドシート保存エラー: {e}")
            print("ヒント: JSONファイル名は合っていますか？ スプシへの招待は完了していますか？")
    else:
        print("保存するデータがありませんでした。")

# === 実行 ===
fetch_and_save_books()

📡 国立国会図書館APIから「〇〇歳の」または「〇〇歳からの」を含む本を収集中...
   検索年齢範囲: 0歳〜100歳

📊 進捗: 1/101歳目 (0歳) - 現在の取得数: 0件
  🔍 「0歳の」で検索中...
    -> 1件目〜 を処理中... (ヒット数: 142)
  🔍 「0歳からの」で検索中...
    -> 1件目〜 を処理中... (ヒット数: 500)

📊 進捗: 2/101歳目 (1歳) - 現在の取得数: 253件
  🔍 「1歳の」で検索中...
    -> 1件目〜 を処理中... (ヒット数: 86)
  🔍 「1歳からの」で検索中...
    -> 1件目〜 を処理中... (ヒット数: 72)

📊 進捗: 3/101歳目 (2歳) - 現在の取得数: 277件
  🔍 「2歳の」で検索中...
    -> 1件目〜 を処理中... (ヒット数: 187)
  🔍 「2歳からの」で検索中...
    -> 1件目〜 を処理中... (ヒット数: 40)

📊 進捗: 4/101歳目 (3歳) - 現在の取得数: 304件
  🔍 「3歳の」で検索中...
    -> 1件目〜 を処理中... (ヒット数: 261)
  🔍 「3歳からの」で検索中...
    -> 1件目〜 を処理中... (ヒット数: 111)

📊 進捗: 5/101歳目 (4歳) - 現在の取得数: 382件
  🔍 「4歳の」で検索中...
    -> 1件目〜 を処理中... (ヒット数: 117)
  🔍 「4歳からの」で検索中...
    -> 1件目〜 を処理中... (ヒット数: 31)

📊 進捗: 6/101歳目 (5歳) - 現在の取得数: 405件
  🔍 「5歳の」で検索中...
    -> 1件目〜 を処理中... (ヒット数: 219)
  🔍 「5歳からの」で検索中...
    -> 1件目〜 を処理中... (ヒット数: 20)

📊 進捗: 7/101歳目 (6歳) - 現在の取得数: 417件
  🔍 「6歳の」で検索中...
    -> 1件目〜 を処理中... (ヒット数: 208)
  🔍 「6歳からの」で検索中...
    -> 1件目〜 を処理中... (ヒット数: 26)

